In [ ]:
import sys
sys.path.append('../../')
from pathlib import Path
import random


import numpy as np
import pandas as pd
from sklearn import metrics, model_selection
import matplotlib.pyplot as plt

plt.ion()   # interactive mode
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from project.infrastructure.model_trainer import Model
from project.infrastructure.img_dataset import ImageDataset
from project.infrastructure.torchsampler import ImbalancedDatasetSampler
import project.infrastructure.utils as utils
import project.infrastructure.pytorch_util as ptu

In [ ]:
# not using for now
try:
    from torchsummary import summary
except ImportError:
    pass

try:
    import seaborn as sns
except ImportError:
    pass

try:
    import optuna
except ImportError:
    pass

In [ ]:
current_dir = Path.cwd()
home_dir = Path.home()
print(f"current_dir: {current_dir}")
print(f"home_dir:{home_dir}")

## Config PATH

In [ ]:
# Config data_dir, img_dir
data_dir = Path("../../data/")
leaf_data_dir: str = "cassava-leaf-disease-classification/"
csv_file_name: str = "train.csv"

csv_file_path = data_dir/leaf_data_dir/csv_file_name

In [ ]:
img_folder_name: str = "train_images"
img_dir = data_dir/leaf_data_dir/img_folder_name

## Load Data description

In [ ]:
df = pd.read_csv(csv_file_path)
print(df.shape)
df.head()

In [ ]:
# Noticed that the dataset is unbalanced
# TODO: handle class weight for imbalanced dataset
df["label"].value_counts()

In [ ]:
# Set seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

if torch.cuda.device_count() > 1:
    torch.cuda.manual_seed_all(SEED)

In [ ]:
# set overflow warning to error instead
np.seterr(all='raise')
torch.autograd.set_detect_anomaly(True)

# Enable cuDNN Auto-tuner before launching training loop
# Improve performance (For convolutional networks only!)
torch.backends.cudnn.benchmark = True

## Split data into train and valid (9:1)

In [ ]:
'''
While I used train_test_split() to create both a training and validation dataset,
consider exploring cross validation instead.
'''
# Split dataset into train and valid
df_train, df_valid = model_selection.train_test_split(
    df,
    test_size=0.1,
    train_size=0.9,
    random_state=SEED,
    stratify=df.label.values
)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)
df_train.shape, df_valid.shape

In [ ]:
df_train.head()

In [ ]:
df_valid.head()

In [ ]:
# Get image path for both training and validation
# Remember to convert path object to str!!
train_img_paths = [str(img_dir/img_id) for img_id in df_train["image_id"].values]
valid_img_paths = [str(img_dir/img_id) for img_id in df_valid["image_id"].values]

# Get image label for both training and validation
train_targets = df_train.label.values
valid_targets = df_valid.label.values

# Verify img paths
train_img_paths[:3], valid_img_paths[:3]

In [ ]:
train_targets

In [ ]:
%%time
# show images
utils.display_image_grid(
    images_filepaths=train_img_paths[0:15],
    predicted_labels=train_targets[0:15]
)

## data_transforms
Image augmentation is a process of creating new training examples from the existing ones.
To make a new sample, you slightly change the original image

* Note that in the validation pipeline we will use A.CenterCrop instead of A.RandomCrop
because we want our validation results to be deterministic
(so that they will not depend upon a random location of a crop).

In [ ]:
# p_i follows p_i = floor(1/i)

p1 = 1      # compose
p2 = 0.5    # operation
p3 = 0.3    # one of
p4 = 0.2    # stand alone or inside one of

# Pleas go through the image dataset to view the image in order to understand what operation need to perform
data_transforms = {
    # Training augmentation
    "train_img_aug": A.Compose(
        [
            # Crop and Resize
            A.Resize(width=300, height=300),
            A.RandomCrop(width=256, height=256),

            # Affine transform
            A.Transpose(p=p2),
            A.HorizontalFlip(p=p2),
            A.VerticalFlip(p=p2),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=p3),

            # Add noise
            A.OneOf([
                A.IAAAdditiveGaussianNoise(p=0.9),
                A.GaussNoise(p=0.6),
            ], p=p3),

            # Blur
            A.OneOf([
                A.MotionBlur(p=.2),
                A.MedianBlur(blur_limit=3, p=0.1),
                A.Blur(blur_limit=3, p=0.1),
            ], p=p3),

            # Distortion
            A.OneOf([
                A.OpticalDistortion(p=0.3),
                A.GridDistortion(p=.1),
                A.IAAPiecewiseAffine(p=0.3),
            ], p=p3),

            # Light intensity
            A.OneOf([
                A.CLAHE(clip_limit=2),
                A.IAASharpen(),
                A.IAAEmboss(),
                A.RandomBrightnessContrast(),
            ], p=p3),

            # RGB
            A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=p2),

            # HSV color space
            A.HueSaturationValue(p=p2),

            # Normalization
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),

        ], p=p1
    ),

    # Validation augmentation
    "valid_img_aug": A.Compose(
        [
            A.Resize(width=300, height=300),
            A.CenterCrop(width=256, height=256),
            A.Transpose(p=p2),
            A.HorizontalFlip(p=p2),
            A.VerticalFlip(p=p2),
            A.ShiftScaleRotate(p=p3),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ], p=p1
    )
}

## Create dataset

In [ ]:
# clear gpu cache to release memory
torch.cuda.empty_cache()

# Create training and validation dataset
train_dataset = ImageDataset(
    image_paths=train_img_paths,
    targets=train_targets,
    augmentations=data_transforms["train_img_aug"]
)

valid_dataset = ImageDataset(
    image_paths=valid_img_paths,
    targets=valid_targets,
    augmentations=data_transforms["valid_img_aug"]
)

print(train_dataset[100], '\n')
# print(type(train_dataset[100]))

## Note dataloader is implemented inside Model.fit()

In [ ]:
%%time
# Visualize training images after augmentation
perm = np.random.permutation(100)[:15]

train_images_array_lst = [train_dataset[int(i)]['image'] for i in perm]
train_images_label_lst = [train_dataset[int(i)]['target'] for i in perm]
utils.display_image_grid(
    images_array_lst=train_images_array_lst,
    true_labels=train_images_label_lst
)

In [ ]:
%%time
# Visualize validation images after augmentation
valid_images_array_lst = [valid_dataset[int(j)]['image'] for j in perm]
valid_images_label_lst = [valid_dataset[int(j)]['target'] for j in perm]
utils.display_image_grid(
    images_array_lst=valid_images_array_lst,
    true_labels=valid_images_label_lst
)


## Transfer Learning fo Computer Vision

Quoting these notes from [cs231n](https://cs231n.github.io/transfer-learning/),

In practice, very few people train an entire Convolutional Network from scratch (with random initialization),
because it is relatively rare to have a dataset of sufficient size.
Instead, it is common to pretrain a ConvNet on a very large dataset
(e.g. ImageNet, which contains 1.2 million images with 1000 categories),
and then use the ConvNet either as an initialization, or a fixed feature extractor for the task of interest.

In [ ]:
class LeafDiseaseClassifier(Model):
    def __init__(self, params: dict,):
        super().__init__()
        self.params = params
        # default: Finetuning the ConvNet
        self.resnet18 = torchvision.models.resnet18(pretrained=params["pretrained"])

        # # As fixed feature extractor
        # for param in self.resnet18.parameters():
        #     param.requires_grad = False

        self.resnet18.fc = nn.Linear(in_features=512, out_features=params["output_size"], bias=True)

    def config_optimizer(self, *args, **kwargs):
        opt = optim.Adam(self.parameters(), lr=self.params["learning_rate"])
        return opt

    # TODO: config lr_scheduler
    # def config_scheduler(self, *args, **kwargs):
    #     assert self.optimizer is not None, "Please set up optimizer first"
    #     sch = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=30, verbose=False)
    #     return sch

    def config_criterion(self, *args, **kwargs):
        criterion = nn.CrossEntropyLoss()
        return criterion

    def loss_fn(self, outputs, targets=None):
        """ calculate loss """
        if targets is None or self.criterion is None:
            print("Targets is None or Criterion is None")
            return None
        return self.criterion(outputs, targets)

    def forward(self, x):
        out: torch.FloatTensor = self.resnet18(x)
        return out

    def monitor_metrics(self, outputs, targets=None) -> dict:
        predictions: np.ndarray = ptu.to_numpy(torch.argmax(outputs, dim=1))
        targets: np.ndarray = ptu.to_numpy(targets)
        accuracy = metrics.accuracy_score(targets, predictions)
        val_metrics = {
            "acc": accuracy,
        }
        return val_metrics

## Define training parameter

In [ ]:
# TODO: change param
print(df.label.unique().shape[0])
# training param
params = {
    "output_size":5,
    "max_epochs": 3,
    "train_batch_size": 8,
    "valid_batch_size": 16*2,
    'fp16': True,
    'seed': 42,
    'no_gpu': False,
    'which_gpu': 0,
    'num_workers': -1,
    'learning_rate': 3e-4,
    'pretrained': True,
    'img_channel': 3,
    'img_height': 256,
    'img_width': 256,
    'save_model': True,

}
assert params["output_size"] == df.label.unique().shape[0]

## Build NN

In [ ]:
# Create ResNet50
resnet18_model:nn.Module = LeafDiseaseClassifier(params)

In [ ]:
# Check if NN build successfully
img = train_dataset[0]["image"]
target = train_dataset[0]["target"]
img, target

In [ ]:
# Build success
resnet18_model(img.unsqueeze(0))


## Start training loop

In [ ]:
# clear gpu cache to release memory
torch.cuda.empty_cache()

In [ ]:
# Init GPU if available
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device("cuda:0")
print(device)



In [ ]:
# init Trainer
resnet18_model.init_trainer(params)

In [ ]:
%%time
# metrics are store in history: dict
history = resnet18_model.fit(
    train_dataset=train_dataset,
    train_batch_size=params["train_batch_size"],
    valid_dataset=valid_dataset,
    valid_batch_size=params["valid_batch_size"],
    max_epochs=params["max_epochs"],
    device=device,
    train_sampler=None,  #ImbalancedDatasetSampler(train_dataset),
    valid_sampler=None,  #ImbalancedDatasetSampler(valid_dataset),
    num_workers=params["num_workers"], use_fp16=params['fp16'],
    save_best=params['save_model'],
    better_than=0.8
                    
)

# plot metrics

In [ ]:
train_losses = history['train_loss']
val_losses = history['val_loss']

plt.plot(train_losses, '-x')
plt.plot(val_losses, '-o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training', 'Validation'])
plt.title('Loss vs. No. of Epochs')
plt.show()



In [ ]:
train_acc = np.array(history['train_acc'])
avg_train_acc = np.vstack(train_acc).mean(axis=1)


val_acc = np.array(history['val_acc'])
avg_val_acc = np.vstack(val_acc).mean(axis=1)

plt.plot(avg_train_acc, '-x')
plt.plot(avg_val_acc, '-o')

plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.title('Accuracy vs. No. of Epochs')
plt.show()

# TODO: add confusion_mtx


## TODO: finish optuna automate hyperparam tuning [example](https://github.com/optuna/optuna/blob/master/examples/pytorch/pytorch_simple.py)

In [ ]:
def define_model(trial: optuna.Trial):
    ...
def objective(trial: optuna.Trial):
    # Generate the model
    model=resnet18_model.to(device)

    # Generate the optimizers
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = ...


In [ ]:
"""
study = optuna.create_study(direction=)
study.optimize(objective(), n_trials=..., timeout=600)
"""

In [ ]:
# clear gpu cache to release memory
torch.cuda.empty_cache()


# TODO: AutoAlbument

